In [ ]:
import os
import torch

import numpy as np
import torch.optim as optim
import torch.nn.functional as F

from torch import nn
from datetime import datetime as dtm
from tqdm import tqdm_notebook as tqdm

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
# Path for save trained models and graphics of training process
SAVE_TO = '../trained_models'

if not os.path.exists(SAVE_TO):
    try:
        os.mkdir(SAVE_TO)
    except FileNotFoundError as e:
        print('ERROR:', e)

In [ ]:
END_TOKEN = '\x04'
TRAIN_PART = 0.65
EPOCHS = 10
HIDDEN_SIZE = 128
NUM_LAYERS = 1
LR = 1e-3
TH = 0.5
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

In [ ]:
normal_1_path = '../data/normal_train.txt'
normal_2_path = '../data/normal_test.txt'
abnormal_path = '../data/abnormal_test.txt'

with open(normal_1_path) as f:
    normal_raw_data = f.readlines()
with open(normal_2_path) as f:
    normal_raw_data += f.readlines()
with open(abnormal_path) as f:
    abnormal_raw_data = f.readlines()

In [ ]:
tokens = set(''.join(normal_raw_data) + ''.join(abnormal_raw_data))
print(len(tokens))
tokens.add(END_TOKEN)
print(len(tokens))
print(tokens)

In [ ]:
int2char = dict(enumerate(tokens))
char2int = {char: idx for idx, char in int2char.items()}

In [ ]:
normal_samples, abnormal_samples = [], []

start_idx = 0
for idx, line in enumerate(normal_raw_data):
    if line == 'Connection: close\n':
        if normal_raw_data[idx + 1] != '\n':
            sample_as_chars = ''.join(normal_raw_data[start_idx:idx + 4]) + END_TOKEN
            start_idx = idx + 5
        else:
            sample_as_chars = ''.join(normal_raw_data[start_idx:idx + 1]) + END_TOKEN
            start_idx = idx + 3
        
        # convert from text to nums
        sample_as_nums = np.array([char2int[char] for char in sample_as_chars])
        normal_samples.append(sample_as_nums)

start_idx = 0 
for idx, line in enumerate(abnormal_raw_data):
    if line == 'Connection: close\n':
        if abnormal_raw_data[idx + 1] != '\n':
            sample_as_chars = ''.join(abnormal_raw_data[start_idx:idx + 4]) + END_TOKEN
            start_idx = idx + 5
        else:
            sample_as_chars = ''.join(abnormal_raw_data[start_idx:idx + 1]) + END_TOKEN
            start_idx = idx + 3
        
        # convert form text to nums
        sample_as_nums = np.array([char2int[char] for char in sample_as_chars])
        abnormal_samples.append(sample_as_nums)

print(len(normal_samples))  # must be 72000
print(len(abnormal_samples))  # must be 25065

In [ ]:
samples = normal_samples + abnormal_samples
labels = np.hstack([np.zeros(len(normal_samples)), np.ones(len(abnormal_samples))])

print(len(samples))
print(len(labels))

In [ ]:
train_indices = []
test_indices = []

for i in np.arange(len(samples)):
    if np.random.uniform() < TRAIN_PART:
        train_indices.append(i)
    else:
        test_indices.append(i)

train_indices = np.array(train_indices)
test_indices = np.array(test_indices)

In [ ]:
def sample2one_hot(sample, size):
    """
    Convert the array of number to one hot vector view.
    
    :param sample: Array of ints.
    :param size: Number of unique nums.
    :return: 2D-array.
    """
    one_hot = np.zeros((len(sample), size), dtype=int)
    one_hot[np.arange(len(sample)), sample] = 1
    return one_hot

In [ ]:
class TrafficRNN(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=1):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
        
        self.h = torch.zeros((num_layers, 1, hidden_size), dtype=torch.float).to(DEVICE)
        self.c = torch.zeros((num_layers, 1, hidden_size), dtype=torch.float).to(DEVICE)
        
    def forward(self, x):
        r_output, (self.h, self.c) = self.lstm(x, (self.h, self.c))
        out = r_output[0, -1, :]
        out = self.fc(out)
        out = self.sigmoid(out)
        
        self.h = torch.zeros((self.num_layers, 1, self.hidden_size), dtype=torch.float).to(DEVICE)
        self.c = torch.zeros((self.num_layers, 1, self.hidden_size), dtype=torch.float).to(DEVICE)
        
        return out

In [ ]:
model = TrafficRNN(len(tokens), HIDDEN_SIZE, NUM_LAYERS)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

model.to(DEVICE)

In [ ]:
best_train_loss, best_test_loss = np.inf, np.inf
best_train_acc, best_test_acc = 0.0, 0.0
train_losses, test_losses = [], []
train_accs, test_accs = [], []

for ep in np.arange(EPOCHS):
    # train
    print('EPOCH #', ep)
    np.random.shuffle(train_indices)
    model.train()
    running_loss = 0.0
    correct = 0
    for i in tqdm(train_indices[:1000]):
        optimizer.zero_grad()

        sample = (sample2one_hot(samples[i], len(tokens))[np.newaxis, :]).astype(np.float32)
        sample = torch.from_numpy(sample).to(DEVICE)
        outputs = model(sample)
        
        predict = 1 if outputs.cpu().detach().numpy()[0] > TH else 0
        label = np.array([labels[i]])
        if (predict == label[0]):
            correct += 1

        label = torch.from_numpy(label.astype(np.float32)).to(DEVICE)
        loss = criterion(outputs, label)
        running_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    
    train_losses.append(running_loss / len(train_indices))
    train_accs.append(correct / len(train_indices))
    
    print('Train loss:', train_losses[-1])
    print('Train acc:', train_accs[-1])
    
    if train_losses[-1] <= best_train_loss:
        best_train_loss = train_losses[-1]
    
    if train_accs[-1] >= best_train_acc:
        best_train_acc = train_accs[-1]

    # test
    with torch.no_grad():
        model.eval()
        running_loss = 0.0
        correct = 0
        for i in tqdm(test_indices[:1000]):
            sample = (sample2one_hot(samples[i], len(tokens))[np.newaxis, :]).astype(np.float32)
            sample = torch.from_numpy(sample).to(DEVICE)
            outputs = model(sample)

            predict = 1 if outputs.cpu().detach().numpy()[0] > TH else 0
            label = np.array([labels[i]])
            if (predict == label[0]):
                correct += 1

            label = torch.from_numpy(label.astype(np.float32)).to(DEVICE)
            loss = criterion(outputs, label)
            running_loss += loss.item()
    
    test_losses.append(running_loss / len(test_indices))
    test_accs.append(correct / len(test_indices))
    
    print('Test loss:', test_losses[-1])
    print('Test acc:', test_accs[-1])
    
    if test_losses[-1] <= best_test_loss:
        best_test_loss = test_losses[-1]
        model_name = dtm.now().strftime('%Y_%m_%d__%H_%M_%S_%f_') + 'loss.pth'
        torch.save({
            'epoch': ep,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': test_losses[-1],
            'accuracy': test_accs[-1]
            }, os.path.join(SAVE_TO, model_name))
    
    if test_accs[-1] >= best_test_acc:
        best_test_acc = test_accs[-1]
        model_name = dtm.now().strftime('%Y_%m_%d__%H_%M_%S_%f_') + 'acc.pth'
        torch.save({
            'epoch': ep,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': test_losses[-1],
            'accuracy': test_accs[-1]
            }, os.path.join(SAVE_TO, model_name))
    
    # plotting
    plt.grid(True)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss dynamic')
    plt.plot(np.arange(len(train_losses)), train_losses, color='orange')
    plt.plot(np.arange(len(test_losses)), test_losses, color='blue')
    plt.legend(('Train', 'Test'))
    plt.savefig(os.path.join(SAVE_TO, 'loss.jpg'), dpi=350)
    plt.clf()
    
    plt.grid(True)
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Accuracy dynamic')
    plt.plot(np.arange(len(train_accs)), train_accs, color='orange')
    plt.plot(np.arange(len(test_accs)), test_accs, color='blue')
    plt.legend(('Train', 'Test'))
    plt.savefig(os.path.join(SAVE_TO, 'acc.jpg'), dpi=350)
    plt.clf()